In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import statistics
import scipy.stats as st
from scipy.stats import pearsonr
from scipy.stats import linregress
import seaborn as sns
import time
start = time.time()
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
filepath='/Data/'
EdgeCutoff=300
Mob=pd.read_csv(filepath+'MSA_Dataset_TillJune.csv',index_col=0)

CtyConfOri=pd.read_csv(filepath+'ArrivalTime/'+'County_ArrT_allrecords_Th1_tillJuly.csv')

CtyConfOri['FIPS']=CtyConfOri['FIPS'].astype(int)
CtyConfOri['Daysfrom122']=CtyConfOri['Days']
sourcelist=list(set(dict.fromkeys(list(Mob['GEOID1']))) & (set(dict.fromkeys(list(CtyConfOri[CtyConfOri['Days']<=53]['FIPS']))))) #
Ctylist=list((set(dict.fromkeys(list(Mob['GEOID1'])))) & (set(dict.fromkeys(list(CtyConfOri['FIPS'])))))
datelistori=list(dict.fromkeys(Mob['date']))[1:]
datelist=datelistori[1:77]
for date in datelist:
    sourcelist=list(set(sourcelist) & set(dict.fromkeys(list(Mob[Mob['date']==date]['GEOID1']))))
R2Rec=pd.DataFrame()
NEdayfrom122=77
cnt=0
for CtySource in sourcelist:
    a=time.time()

    cnt=cnt+1
    CtyTargetlist=[i for i in Ctylist if i!=CtySource] 
    CtyDayZero=CtyConfOri[CtyConfOri['FIPS']==CtySource]['Daysfrom122'].iat[0]
    CtyConfOri['Days']=CtyConfOri['Daysfrom122']-CtyDayZero
    Ctydatelist=datelistori[int(CtyDayZero):]
    CountyEffdistRecALLSingle=pd.DataFrame()
    Days=0
    targetlist=CtyTargetlist
    for date in Ctydatelist:
        DayMob=pd.DataFrame()
        DayMob=Mob[Mob['date']==date]
        DayMob['Fraction']=DayMob['counts_x']/DayMob['sum_x'] #sum_x
        DayMob['effdist']=1-np.log(DayMob['Fraction'])
        G=nx.from_pandas_edgelist(DayMob,source='GEOID1',target='GEOID2',edge_attr='effdist',create_using=nx.DiGraph())
        d={'Target':targetlist}
        dftemp=pd.DataFrame(d)
        try:
            length,path = nx.single_source_dijkstra(G,CtySource,cutoff=EdgeCutoff,weight='effdist')
            dftemp['CapD']=dftemp['Target'].map(length)
            dftemp['Date']=date
            dftemp['Days']=Days
            dftemp['Route']=dftemp['Target'].map(path)
            dftemp=dftemp.dropna(subset=['Route'])
            dftemp['Nodes']=dftemp['Route'].apply(lambda x: len(x))
            CountyEffdistRecALLSingle=CountyEffdistRecALLSingle.append(pd.DataFrame(dftemp))
            Days=Days+1
        except:
            Days=Days+1

    CtyNEday=NEdayfrom122-CtyDayZero
    CountyEffdistRecALL=CountyEffdistRecALLSingle
    ArrTRec=CtyConfOri[['FIPS','Days']].set_index('FIPS').to_dict('dict')
    CountyEffdistRecALL['ArrT']=CountyEffdistRecALL['Target'].map(ArrTRec['Days'])
    CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['ArrT'])
    CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['CapD'])
    CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['Nodes'])
    WindowLenCty=0
    CtyTDRec=CountyEffdistRecALL

    CtyTDRec=CtyTDRec[(CtyTDRec['Days']<=CtyTDRec['ArrT']) & (CtyTDRec['Days']>=CtyTDRec['ArrT']-WindowLenCty)]


    CtyTDRec=CtyTDRec.groupby(['Target']).mean()
    CtyTDRec=CtyTDRec.reset_index()


    CtyTDRec=CtyTDRec[CtyTDRec['ArrT']<=CtyNEday]

    TDRecDNA=CtyTDRec.dropna(subset=['CapD','ArrT'])
    linregress(TDRecDNA['CapD'],TDRecDNA['ArrT'])


    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(TDRecDNA[['CapD']], TDRecDNA[['ArrT']])

    # Make predictions using the testing set
    diabetes_y_pred = regr.predict(TDRecDNA[['CapD']])
    r2=r2_score(TDRecDNA[['ArrT']], diabetes_y_pred)
    R2Rec=R2Rec.append(pd.DataFrame({'Source':CtySource,'R2':r2},index=[0]),ignore_index=True)
    b=time.time()
    
    print(str(cnt/len(sourcelist))+' Time:'+str(b-a),end='\r')

In [ ]:
R2Rec.to_csv('/SingleSource_FinalResult_EffDist_R2Rec.csv')

In [ ]:
R2Rec.sort_values(by=['R2'], ascending=False).head(10)

In [ ]:
from scipy import stats

def getPairStats(x, y):
    
    #calculate means
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    #get number of entries
    n = len(x)
    
    #calculate sums
    x_sum = np.sum(x)
    x_sum_square = np.sum([xi**2 for xi in x])
    y_sum = np.sum(y)
    y_sum_square = np.sum([yi**2 for yi in y])
    xy_sum = np.sum([xi*yi for xi, yi in zip(x, y)])
    
    #calculcate remainder of equations
    s_xx  = x_sum_square - (1/n)*(x_sum**2)
    s_yy = y_sum_square - (1/n)*(y_sum**2)
    s_xy = xy_sum - (1/n)*x_sum*y_sum
    
    return s_xx, s_yy, s_xy
def pair_regression(x, y):
    """
    perform simple regression between predictor variable x and target variable y
    """
    
    #calculate means
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    #get number of entries
    n = len(x)
    
    #get stats
    s_xx, s_yy, s_xy = getPairStats(x, y)
    
    #calculcate coefficients
    beta_hat = s_xy / s_xx
    alpa_hat = y_bar - beta_hat * x_bar
    
    return alpa_hat, beta_hat
def simple_regression_conf(x, y, alpha=0.05):
    
    n = len(x)
    #calculate stats
    s_xx, s_yy, s_xy = getPairStats(x, y)
    
    #get regression coefficients
    alpha_hat, beta_hat = pair_regression(x, y)
    
    #maximim likelihood estimator
    sigma_hat = np.sqrt((1/n)*(s_yy-beta_hat*s_xy))
    
    #calculcate t statistic
    t_limit = stats.t.ppf(1-alpha/2, n-2)
    #interval term
    interval_val = t_limit*sigma_hat*np.sqrt(n/((n-2)*s_xx))
    
    #calculcate intervals
    beta_upper = beta_hat + interval_val
    beta_lower = beta_hat - interval_val
    
    return beta_lower, beta_upper

beta_lower, beta_upper = simple_regression_conf(x=TDRecDNA['CapD'], y=TDRecDNA['ArrT'], alpha=0.05)
print("beta_lower: {:.4f}".format(beta_lower))
print("beta_upper: {:.4f}".format(beta_upper))